In [1]:
%load_ext autoreload

In [2]:
# pl packages
import argparse
import torch
from pytorch_lightning.utilities.seed import seed_everything

# python packages
import os
import pandas as pd

os.chdir('../')
import sys

root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

# single cell packages
import scanpy as sc
from scvi import REGISTRY_KEYS

Global seed set to 0


In [3]:
%autoreload 2
# own packages
from feature_attribution_sc.models.scgen_models import SCGENCustom

In [4]:
batch_size=2048
feature_importance='/home/icb/till.richter/git/feature-attribution-sc/outputs/baselines/task1_random.csv'
threshold=0.5
data_path='/home/icb/yuge.ji/projects/feature-attribution-sc/'
CHECKPOINT_PATH='/home/icb/till.richter/git/feature-attribution-sc/trained_models/scgen/'
max_epochs=500
early_stopping=True
patience=10

In [5]:
# GET GPU AND ARGS
if torch.cuda.is_available():
    print(f'CUDA_VISIBLE_DEVICES: {os.environ["CUDA_VISIBLE_DEVICES"]}')

In [6]:
# GET DATA
adata = sc.read(f'{data_path}/datasets/scgen_norman19.h5ad')

In [7]:
# FIX SEED FOR REPRODUCIBILITY
seed_everything(90)

Global seed set to 90


90

In [8]:
# GET MODEL
SCGENCustom.setup_anndata(adata)

In [9]:
model = SCGENCustom(adata, feature_importance=pd.read_csv(feature_importance), threshold=threshold)

In [10]:
# CHECKPOINT CALLBACK
CHECKPOINT_PATH = CHECKPOINT_PATH + feature_importance.split('/')[-1].split('.')[
    0] + '_' + str(threshold) + '_checkpoints.pt'  # set checkpoint path
print(f'CHECKPOINT_PATH: {CHECKPOINT_PATH}')

model.save(CHECKPOINT_PATH, overwrite=True)  # save model to checkpoint

CHECKPOINT_PATH: /home/icb/till.richter/git/feature-attribution-sc/trained_models/scgen/task1_random_0.5_checkpoints.pt


In [11]:
# TRAIN
model.train(
    max_epochs=max_epochs,
    batch_size=batch_size,
    early_stopping=early_stopping,
    early_stopping_patience=patience
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 2/500:   0%|▎                                                                                                                                                              | 1/500 [02:17<19:07:28, 137.97s/it, loss=841, v_num=1]